In [10]:
install.packages("datateachr")
library(tidyverse)
library(infer)
library(gridExtra)
library(testthat)
library(digest)
library(broom)

Installing package into ‘/home/jupyter/R/x86_64-pc-linux-gnu-library/4.1’
(as ‘lib’ is unspecified)

Warning message:
“package ‘datateachr’ is not available for this version of R

A version of this package for your version of R might be available elsewhere,
see the ideas at
https://cran.r-project.org/doc/manuals/r-patched/R-admin.html#Installing-packages”


In [11]:
stars <- read.csv("6-class.csv")

head(stars)

,Temperature..K.,Luminosity.L.Lo.,Radius.R.Ro.,Absolute.magnitude.Mv.,Star.type,Star.color,Spectral.Class
,<int>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<chr>
1,3068,0.002400,0.1700,16.12,0,Red,M
2,3042,0.000500,0.1542,16.60,0,Red,M
3,2600,0.000300,0.1020,18.70,0,Red,M
4,2800,0.000200,0.1600,16.65,0,Red,M
5,1939,0.000138,0.1030,20.06,0,Red,M
6,2840,0.000650,0.1100,16.98,0,Red,M


In [12]:
stars <- mutate(stars, Star.type = as_factor(Star.type))
dwarfs <- stars %>%
filter(Star.type != 5) %>%
filter(Star.type != 4) %>%
filter(Star.type != 3)

head(dwarfs)

,Temperature..K.,Luminosity.L.Lo.,Radius.R.Ro.,Absolute.magnitude.Mv.,Star.type,Star.color,Spectral.Class
,<int>,<dbl>,<dbl>,<dbl>,<fct>,<chr>,<chr>
1,3068,0.002400,0.1700,16.12,0,Red,M
2,3042,0.000500,0.1542,16.60,0,Red,M
3,2600,0.000300,0.1020,18.70,0,Red,M
4,2800,0.000200,0.1600,16.65,0,Red,M
5,1939,0.000138,0.1030,20.06,0,Red,M
6,2840,0.000650,0.1100,16.98,0,Red,M


In [13]:
dwarfs_temp <- dwarfs %>%
               select(Temperature..K., Star.type)
head(dwarfs_temp)

,Temperature..K.,Star.type
,<int>,<fct>
1,3068,0
2,3042,0
3,2600,0
4,2800,0
5,1939,0
6,2840,0


In [14]:
anova_temp <- aov(Temperature..K.~Star.type, data = dwarfs_temp)%>%
              tidy()
print(anova_temp)

# A tibble: 2 × 6
  term         df       sumsq      meansq statistic   p.value
  <chr>     <dbl>       <dbl>       <dbl>     <dbl>     <dbl>
1 Star.type     2 3106600865. 1553300433.      188.  2.74e-37
2 Residuals   117  965696814.    8253819.       NA  NA       


In [15]:
f_stat_temp <- anova_temp %>%
          filter(term == "Star.type") %>%
          select(statistic) %>%
          as.numeric()
anova_pval_temp <- anova_temp %>%
              filter(term == "Star.type") %>%
              select(p.value) %>%
              as.numeric()
print(f_stat_temp)
print(anova_pval_temp)

[1] 188.1917
[1] 2.738892e-37


In [16]:
dwarfs_lumi <- dwarfs %>%
               select(Luminosity.L.Lo., Star.type)
head(dwarfs_lumi)

,Luminosity.L.Lo.,Star.type
,<dbl>,<fct>
1,0.002400,0
2,0.000500,0
3,0.000300,0
4,0.000200,0
5,0.000138,0
6,0.000650,0


In [17]:
anova_lumi <- aov(Luminosity.L.Lo.~Star.type, data = dwarfs_lumi)%>%
              tidy()
print(anova_lumi)

# A tibble: 2 × 6
  term         df    sumsq    meansq statistic  p.value
  <chr>     <dbl>    <dbl>     <dbl>     <dbl>    <dbl>
1 Star.type     2 0.000454 0.000227       5.09  0.00761
2 Residuals   117 0.00522  0.0000447     NA    NA      


In [18]:
f_stat_lumi <- anova_lumi %>%
          filter(term == "Star.type") %>%
          select(statistic) %>%
          as.numeric()
anova_pval_lumi <- anova_lumi%>%
              filter(term == "Star.type") %>%
              select(p.value) %>%
              as.numeric()
print(f_stat_lumi)
print(anova_pval_lumi)

[1] 5.086809
[1] 0.007614593
